## 1. Setup and Installation

In [2]:
# Import required modules
import os
import sys
from pathlib import Path

# Add the project root to the path
project_root = Path().resolve().parent
sys.path.insert(0, str(project_root))

print(f"Project root: {project_root}")

Project root: /Users/gphome/Desktop/projects/thesis-ds/feedback-loop


In [3]:
# Import the SimLP client
from src.simlp import SimLPClient
from src.core.models import EvaluationResult

print("✓ Imports successful!")

✓ Imports successful!


## 2. Basic Evaluation

Let's start with a simple example: evaluating a generated rule against a reference.

### 2.1 Perfect Match Example

In [4]:
# Initialize the SimLP client
client = SimLPClient()

# Define reference rules (ground truth)
reference_rules = """
initiatedAt(gap(Vessel)=nearPorts, T) :-
    happensAt(gap_start(Vessel), T),
    holdsAt(withinArea(Vessel, nearPorts)=true, T).

initiatedAt(gap(Vessel)=farFromPorts, T) :-
    happensAt(gap_start(Vessel), T),
    not holdsAt(withinArea(Vessel, nearPorts)=true, T).

terminatedAt(gap(Vessel)=_Status, T) :-
    happensAt(gap_end(Vessel), T).
"""

# Generated rules that perfectly match
generated_rules ="""
initiatedAt(gap(Vessel)=nearPorts, T) :-
    happensAt(gap_start(Vessel), T),
    holdsAt(withinArea(Vessel, nearPorts)=true, T).

initiatedAt(gap(Vessel)=farFromPorts, T) :-
    happensAt(gap_start(Vessel), T),
    not holdsAt(withinArea(Vessel, nearPorts)=true, T).

terminatedAt(gap(Vessel)=_Status, T) :-
    happensAt(gap_end(Vessel), T).
"""

print("Reference rules:")
print(reference_rules)
print("\n" + "="*80 + "\n")

Reference rules:

initiatedAt(gap(Vessel)=nearPorts, T) :-
    happensAt(gap_start(Vessel), T),
    holdsAt(withinArea(Vessel, nearPorts)=true, T).

initiatedAt(gap(Vessel)=farFromPorts, T) :-
    happensAt(gap_start(Vessel), T),
    not holdsAt(withinArea(Vessel, nearPorts)=true, T).

terminatedAt(gap(Vessel)=_Status, T) :-
    happensAt(gap_end(Vessel), T).





In [5]:
# Evaluate the rules
result = client.evaluate(
    domain='MSA',
    activity='gap',
    generated_rules=generated_rules,
    reference_rules=reference_rules,
    generate_feedback=True
)

# Display results
print(f"Rule ID: {result.rule_id}")
print(f"Similarity Score: {result.score:.4f}")
print(f"Matches Reference: {result.matches_reference}")
print(f"\nNumber of Issues: {len(result.issues)}")
if result.issues:
    print("Issues:")
    for issue in result.issues:
        print(f"  - {issue}")
else:
    print("✓ No issues found!")

Concepts defined in both event descriptions: 
[('gap', 'initiatedAt'), ('gap', 'terminatedAt')]

Concepts defined only in generated event description: 
[]

Concepts defined only in ground event description: 
[]

Similarity for definition: ('gap', 'initiatedAt') is 1.0
Similarity for definition: ('gap', 'terminatedAt') is 1.0
Event Description Similarity is: 
1.0
Rule ID: MSA_gap
Similarity Score: 1.0000
Matches Reference: True

Number of Issues: 0
✓ No issues found!


### 2.2 Incomplete Rules Example

What happens when rules are incomplete?

In [ ]:
# Generated rules with missing termination condition
incomplete_rules = """
initiatedAt(gap(Vessel)=nearPorts, T) :-
    happensAt(gap_start(Vessel), T),
    holdsAt(withinArea(Vessel, nearPorts)=true, T).
"""

print("Incomplete rules (missing farFromPorts and terminatedAt):")
print(incomplete_rules)

In [ ]:
# Evaluate incomplete rules
result_incomplete = client.evaluate(
    domain='MSA',
    activity='gap',
    generated_rules=incomplete_rules,
    reference_rules=reference_rules,
    generate_feedback=True
)

print(f"Similarity Score: {result_incomplete.score:.4f}")
print(f"Matches Reference: {result_incomplete.matches_reference}")
print(f"\nIssues Found: {len(result_incomplete.issues)}")
for i, issue in enumerate(result_incomplete.issues, 1):
    print(f"  {i}. {issue}")

print(f"\n{'='*80}")
print("Feedback:")
print("="*80)
print(result_incomplete.feedback)

### 2.3 Incorrect Rules Example

What about rules with logical errors?

In [ ]:
# Rules with incorrect predicate
incorrect_rules = """
initiatedAt(gap(Vessel)=nearPorts, T) :-
    happensAt(gap_start(Vessel), T),
    holdsAt(nearCoast(Vessel)=true, T).

initiatedAt(gap(Vessel)=farFromPorts, T) :-
    happensAt(gap_start(Vessel), T),
    not holdsAt(nearCoast(Vessel)=true, T).

terminatedAt(gap(Vessel)=_Status, T) :-
    happensAt(gap_end(Vessel), T).
"""

print("Incorrect rules (using 'nearCoast' instead of 'withinArea'):")
print(incorrect_rules)

In [ ]:
# Evaluate incorrect rules
result_incorrect = client.evaluate(
    domain='MSA',
    activity='gap',
    generated_rules=incorrect_rules,
    reference_rules=reference_rules,
    generate_feedback=True
)

print(f"Similarity Score: {result_incorrect.score:.4f}")
print(f"Matches Reference: {result_incorrect.matches_reference}")
print(f"\nNote: Even structurally similar rules get lower scores if predicates differ")

## 3. Understanding Similarity Scores

Let's visualize how similarity scores relate to rule quality.

In [ ]:
import pandas as pd

# Collect results from different rule variants
test_cases = [
    ('Perfect Match', generated_rules, 1.0),
    ('Incomplete (1/3 rules)', incomplete_rules, result_incomplete.score),
    ('Incorrect Predicates', incorrect_rules, result_incorrect.score),
]

# Create DataFrame
df = pd.DataFrame(test_cases, columns=['Scenario', 'Rules', 'Score'])
df['Matches'] = df['Score'] >= 0.9
df['Category'] = pd.cut(
    df['Score'],
    bins=[0, 0.5, 0.9, 1.0],
    labels=['Low', 'Moderate', 'High']
)

print("Similarity Score Analysis:")
print("="*80)
print(df[['Scenario', 'Score', 'Matches', 'Category']].to_string(index=False))
print("\n" + "="*80)
print("Score Interpretation:")
print("  >= 0.9 (High):     Rules match reference - ready to use")
print("  0.5 - 0.9 (Moderate): Minor improvements needed")
print("  < 0.5 (Low):       Significant differences - major revision required")

## 4. Working with Reference Files

In practice, reference rules are stored in files. Let's create a temporary reference structure.

In [ ]:
import tempfile
from pathlib import Path

# Create temporary directory structure
temp_dir = Path(tempfile.mkdtemp())
ref_dir = temp_dir / "references" / "MSA"
ref_dir.mkdir(parents=True)

# Write reference rules to file
gap_ref_file = ref_dir / "gap.pl"
gap_ref_file.write_text(reference_rules)

print(f"Created reference directory: {ref_dir}")
print(f"Reference file: {gap_ref_file}")
print(f"File exists: {gap_ref_file.exists()}")

In [ ]:
# Initialize client with reference directory
client_with_refs = SimLPClient(
    reference_rules_dir=str(temp_dir / "references"),
    log_dir=str(temp_dir / "logs")
)

# Evaluate without providing reference_rules parameter
# (will load from file automatically)
result_from_file = client_with_refs.evaluate(
    domain='MSA',
    activity='gap',
    generated_rules=incomplete_rules,
    generate_feedback=True
)

print(f"Loaded reference from file: {gap_ref_file.name}")
print(f"Similarity Score: {result_from_file.score:.4f}")
print(f"\nLog file created at: {result_from_file.metadata['log_file']}")

## 5. Detailed Feedback Analysis

Let's explore the structured feedback data.

In [ ]:
# Access metadata
metadata = result_incomplete.metadata

print("Evaluation Metadata:")
print("="*80)
print(f"Domain: {metadata['domain']}")
print(f"Activity: {metadata['activity']}")
print(f"Log file: {metadata['log_file']}")
print(f"\nOptimal matching: {metadata['optimal_matching']}")
print(f"Distances: {metadata['distances']}")
print(f"\nFeedback data available: {bool(metadata['feedback_data'])}")

In [ ]:
# Examine feedback data structure
feedback_data = metadata['feedback_data']

if feedback_data:
    print("Feedback Data Structure:")
    print("="*80)
    for concept, data in feedback_data.items():
        print(f"\nConcept: {concept}")
        print("-" * 40)
        if isinstance(data, dict):
            for key, value in data.items():
                if isinstance(value, list):
                    print(f"  {key}: {len(value)} items")
                    for item in value[:3]:  # Show first 3
                        print(f"    - {item}")
                else:
                    print(f"  {key}: {value}")
else:
    print("No detailed feedback data available")

## 6. Integration with Feedback Loop

Here's how SimLP evaluation integrates with the complete feedback loop.

In [ ]:
from src.prompts.builder import MSAPromptBuilder

# Simulated feedback loop iteration
def simulate_feedback_iteration(activity, initial_rules, reference_rules):
    """
    Simulate one iteration of the feedback loop.
    """
    print(f"\n{'='*80}")
    print(f"Evaluating activity: {activity}")
    print("="*80)
    
    # Evaluate rules
    result = client.evaluate(
        domain='MSA',
        activity=activity,
        generated_rules=initial_rules,
        reference_rules=reference_rules,
        generate_feedback=True
    )
    
    print(f"\nSimilarity Score: {result.score:.4f}")
    print(f"Matches Reference: {result.matches_reference}")
    
    # Check if refinement needed
    if not result.matches_reference:
        print("\n⚠️  Refinement needed!")
        print(f"\nIssues ({len(result.issues)}):")
        for i, issue in enumerate(result.issues, 1):
            print(f"  {i}. {issue}")
        
        print("\n📋 Feedback for LLM:")
        print("-" * 80)
        print(result.feedback[:500] + "..." if len(result.feedback) > 500 else result.feedback)
        
        # In real implementation, this would go to build_refinement()
        print("\n→ Next: Pass feedback to PromptBuilder.build_refinement()")
    else:
        print("\n✓ Rules accepted! No refinement needed.")
    
    return result

# Run simulation
result = simulate_feedback_iteration(
    activity='gap',
    initial_rules=incomplete_rules,
    reference_rules=reference_rules
)